# Loss Functions

In [40]:
%cd ../..

/home/karimgamaleldin


In [41]:
# imports 
from nn import BCELoss, CrossEntropyLoss, MSELoss, BCEWithLogitsLoss, Linear, ReLU, Sigmoid, Softmax
from core import KTorch
from autograd import Tensor
import numpy as np
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

### MSE Loss

In [42]:
# Get inputs
np.random.seed(0)

y_preds = np.random.randint(100, 1000, (32, ))
y_true = np.random.randint(100, 1000, (32, ))
y_preds == y_true

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [43]:
y_preds_tensor = Tensor(y_preds)
y_true_tensor = Tensor(y_true)

# MSE Loss
mse_loss_sum = MSELoss(reduction='sum')
mse_loss_mean = MSELoss(reduction='mean')

# Calculate loss
loss_sum = mse_loss_sum(y_preds_tensor, y_true_tensor)
loss_mean = mse_loss_mean(y_preds_tensor, y_true_tensor)

print(f"Sum Loss: {loss_sum}")
print(f"Mean Loss: {loss_mean}")

Sum Loss: tensor: 3806477.0
Mean Loss: tensor: 118952.40625


In [44]:
# Calculate mse loss manual
diff = y_preds - y_true
mse_mean = np.mean(diff**2)
mse_sum = np.sum(diff**2)

print(f"Manual Sum Loss: {mse_sum}")
print(f"Manual Mean Loss: {mse_mean}")

Manual Sum Loss: 3806477
Manual Mean Loss: 118952.40625


In [45]:
# Testing backprop

# input 
np.random.seed(0)
input = np.random.randn(32, 10).astype(np.float32)
target = np.random.rand(32, 1).astype(np.float32)

# Convert to tensors
x = Tensor(input)
y = Tensor(target)

# model
linear_1 = Linear(10, 5)
relu = ReLU()
linear_2 = Linear(5, 1)
relu_2 = ReLU()

# forward pass
h1_nn = linear_1(x)
h2_nn = relu(h1_nn)
h3_nn = linear_2(h2_nn)
out = relu_2(h3_nn)

out.shape

(32, 1)

In [46]:
# Manual forward pass
weights_1 = linear_1.weight.data
bias_1 = linear_1.bias.data
weights_2 = linear_2.weight.data
bias_2 = linear_2.bias.data

h1 = np.dot(input, weights_1) + bias_1
h2 = np.maximum(h1, 0)
h3 = np.dot(h2, weights_2) + bias_2
out_manual = np.maximum(h3, 0)

out_manual.shape

(32, 1)

In [47]:
np.equal(out.data, out_manual).all()

True

In [48]:
# Loss
loss = MSELoss(reduction='mean')
loss_val = loss(out, y)

# Manual loss
diff = out_manual - target
loss_manual = np.mean(diff**2)

np.equal(loss_val.data, loss_manual).all()

True

In [49]:
# Backward pass
loss_val.backward()

In [50]:
# Manual backward pass
dL = np.ones_like(diff)
print((loss_val.grad == dL).all())
d_diff = dL * 2 * diff / 32
d_out_manual = d_diff
print((d_out_manual == out.grad).all())
d_h3 = d_out_manual * (h3 > 0)
print((d_h3 == h3_nn.grad).all())
d_weights_2 = np.dot(h2.T, d_h3)
d_bias_2 = np.sum(d_h3, axis=0)
print((d_weights_2 == linear_2.weight.grad).all())
print((d_bias_2 == linear_2.bias.grad).all())
d_h2 = np.dot(d_h3, weights_2.T)
print((d_h2 == h2_nn.grad).all())
d_h1 = d_h2 * (h1 > 0)
print((d_h1 == h1_nn.grad).all())
d_weights_1 = np.dot(input.T, d_h1)
d_bias_1 = np.sum(d_h1, axis=0)
print((d_weights_1 == linear_1.weight.grad).all())
print((d_bias_1 == linear_1.bias.grad).all())

True
True
True
True
True
True
True
True
True


#### MSE Sum

In [51]:
# Testing backprop

# input 
np.random.seed(0)
input = np.random.randn(32, 10).astype(np.float32)
target = np.random.rand(32, 1).astype(np.float32)

# Convert to tensors
x = Tensor(input)
y = Tensor(target)

# model
linear_1 = Linear(10, 5)
relu = ReLU()
linear_2 = Linear(5, 1)
relu_2 = ReLU()

# forward pass
h1_nn = linear_1(x)
h2_nn = relu(h1_nn)
h3_nn = linear_2(h2_nn)
out = relu_2(h3_nn)

out.shape

(32, 1)

In [52]:
# Manual forward pass
weights_1 = linear_1.weight.data
bias_1 = linear_1.bias.data
weights_2 = linear_2.weight.data
bias_2 = linear_2.bias.data

h1 = np.dot(input, weights_1) + bias_1
h2 = np.maximum(h1, 0)
h3 = np.dot(h2, weights_2) + bias_2
out_manual = np.maximum(h3, 0)

out_manual.shape

(32, 1)

In [53]:
np.equal(out.data, out_manual).all()

True

In [54]:
# Loss
loss = MSELoss(reduction='sum')
loss_val = loss(out, y)

# Manual loss
diff = out_manual - target
loss_manual = np.sum(diff**2)

np.equal(loss_val.data, loss_manual).all()

True

In [55]:
# Backward pass
loss_val.backward()

In [56]:
# Manual backward pass
dL = np.ones_like(diff)
print((loss_val.grad == dL).all())
d_diff = dL * 2 * diff
d_out_manual = d_diff
print((d_out_manual == out.grad).all())
d_h3 = d_out_manual * (h3 > 0)
print((d_h3 == h3_nn.grad).all())
d_weights_2 = np.dot(h2.T, d_h3)
d_bias_2 = np.sum(d_h3, axis=0)
print((d_weights_2 == linear_2.weight.grad).all())
print((d_bias_2 == linear_2.bias.grad).all())
d_h2 = np.dot(d_h3, weights_2.T)
print((d_h2 == h2_nn.grad).all())
d_h1 = d_h2 * (h1 > 0)
print((d_h1 == h1_nn.grad).all())
d_weights_1 = np.dot(input.T, d_h1)
d_bias_1 = np.sum(d_h1, axis=0)
print((d_weights_1 == linear_1.weight.grad).all())
print((d_bias_1 == linear_1.bias.grad).all())

True
True
True
True
True
True
True
True
True


### BCELoss

In [57]:
# Get inputs
np.random.seed(0)
inputs = np.random.randn(32, 10).astype(np.float32)
targets = np.random.randint(0, 2, (32, 1)).astype(np.float32)

# Convert to tensors
x = Tensor(inputs)
y = Tensor(targets)

# Try forward pass of bce loss
preds = np.random.rand(32, 1).astype(np.float32)
preds_tensor = Tensor(preds)

bce_loss_mean = BCELoss(reduction='mean')
bce_loss_sum = BCELoss(reduction='sum')

loss_mean = bce_loss_mean(preds_tensor, y)
loss_sum = bce_loss_sum(preds_tensor, y)

print(f"Mean Loss: {loss_mean}")
print(f"Sum Loss: {loss_sum}")

loss_mean.data.dtype

Mean Loss: tensor: 1.1908915042877197
Sum Loss: tensor: 38.10852813720703


dtype('float32')

In [58]:
# Manual loss calculation
log_1 = np.log(preds).astype(np.float32)
log_2 = np.log(1 - preds).astype(np.float32)
log_1 = np.clip(log_1, -100, float('inf'))
log_2 = np.clip(log_2, -100, float('inf'))
loss_term_1 = - targets * log_1
loss_term_2 = - (1 - targets) * log_2
loss = loss_term_1 + loss_term_2
loss_manual_mean = np.mean(loss)
loss_manual_sum = np.sum(loss)

print(f"Manual Mean Loss: {loss_manual_mean}")
print(f"Manual Sum Loss: {loss_manual_sum}")

Manual Mean Loss: 1.1908915042877197
Manual Sum Loss: 38.10852813720703


In [59]:
# Check loss values
np.equal(loss_mean.data, loss_manual_mean), np.equal(loss_sum.data, loss_manual_sum)

(True, True)

In [60]:
# Forward pass
np.random.seed(0)

layer_1 = Linear(10, 5)
relu = ReLU()
layer_2 = Linear(5, 1)
sigmoid = Sigmoid()

# Forward pass
h1 = layer_1(x)
h2 = relu(h1)
h3 = layer_2(h2)
out = sigmoid(h3)

out.shape

(32, 1)

In [61]:
# Manual forward pass
weights_1 = layer_1.weight.data
bias_1 = layer_1.bias.data
weights_2 = layer_2.weight.data
bias_2 = layer_2.bias.data

h1_manual = np.dot(inputs, weights_1) + bias_1
h2_manual = np.maximum(h1_manual, 0)
h3_manual = np.dot(h2_manual, weights_2) + bias_2
out_manual = 1 / (1 + np.exp(-h3_manual))

out_manual.shape

(32, 1)

In [62]:
# Check if the outputs are the same
np.equal(out.data, out_manual).all()

True

In [63]:
# Loss
bce_loss = BCELoss(reduction='mean')
loss_val = bce_loss(out, y)

# Manual loss
log_1 = np.log(out_manual).astype(np.float32)
sub_out_manual = 1 - out_manual
log_2 = np.log(sub_out_manual).astype(np.float32)
log_1_clip = np.clip(log_1, -100, float('inf'))
log_2_clip = np.clip(log_2, -100, float('inf'))
loss_term_1 = - targets * log_1_clip
loss_term_2 = - (1 - targets) * log_2_clip
loss = loss_term_1 + loss_term_2
loss_manual = np.mean(loss)

np.equal(loss_val.data, loss_manual).all()

True

In [64]:
# Backward pass
loss_val.backward()

In [65]:
# Manual backward pass
dL = np.ones_like(loss)
print((loss_val.grad == dL).all())

d_loss = dL / 32
print((d_loss == loss_val._prev[0].grad).all(), loss_val._prev[0].label)

d_loss_term_1 = d_loss
print((d_loss_term_1 == loss_val._prev[0]._prev[0].grad).all()) 

d_loss_term_2 = d_loss
print((d_loss_term_1 == loss_val._prev[0]._prev[1].grad).all()) 

d_log_1_clip = - targets * d_loss_term_1
print((d_log_1_clip == loss_val._prev[0]._prev[0]._prev[1].grad).all(), loss_val._prev[0]._prev[0]._prev[1].label) 

d_log_2_clip = d_loss_term_2 * - (1 - targets)
print((d_log_2_clip == loss_val._prev[0]._prev[1]._prev[1].grad).all(), loss_val._prev[0]._prev[1]._prev[1].label)

d_log_1  = d_log_1_clip * (log_1 > -100)
print((d_log_1 == loss_val._prev[0]._prev[0]._prev[1]._prev[0].grad).all(), loss_val._prev[0]._prev[0]._prev[1]._prev[0].label)

d_log_2 = d_log_2_clip * (log_2 > -100)
print((d_log_2 == loss_val._prev[0]._prev[1]._prev[1]._prev[0].grad).all(), loss_val._prev[0]._prev[1]._prev[1]._prev[0].label)

d_out_manual = d_log_1  / out_manual
d_sub_out_manual = d_log_2 / sub_out_manual
d_out_manual = d_out_manual - d_sub_out_manual
print((d_out_manual == out.grad).all(), out.label)

d_h3_manual = d_out_manual * out_manual * (1 - out_manual)  
print((d_h3_manual == h3.grad).all())

d_weights_2 = np.dot(h2_manual.T, d_h3_manual)
d_bias_2 = np.sum(d_h3_manual, axis=0)
print((d_weights_2 == layer_2.weight.grad).all())

d_h2_manual = np.dot(d_h3_manual, weights_2.T)
print((d_h2_manual == h2.grad).all())

d_h1_manual = d_h2_manual * (h1_manual > 0)
print((d_h1_manual == h1.grad).all())

d_weights_1 = np.dot(inputs.T, d_h1_manual)
d_bias_1 = np.sum(d_h1_manual, axis=0)
print((d_weights_1 == layer_1.weight.grad).all())
print((d_bias_1 == layer_1.bias.grad).all())

True
True  *  * clamp(log(sigmoid(ReLU( @  + ) @  + ))) +  *  +  *  * clamp(log(sigmoid(ReLU( @  + ) @  + ) *  + ))
True
True
True clamp(log(sigmoid(ReLU( @  + ) @  + )))
True clamp(log(sigmoid(ReLU( @  + ) @  + ) *  + ))
True log(sigmoid(ReLU( @  + ) @  + ))
True log(sigmoid(ReLU( @  + ) @  + ) *  + )
True sigmoid(ReLU( @  + ) @  + )
True
True
True
True
True
True


### BCEWithLogitsLoss

In [66]:
# Get inputs
np.random.seed(0)
inputs = np.random.randn(32, 10).astype(np.float32)
targets = np.random.randint(0, 2, (32, 1)).astype(np.float32)

# Convert to tensors
x = Tensor(inputs)
y = Tensor(targets)

# Try forward pass of bce loss
preds = np.random.randint(-100, 100, (32, 1)).astype(np.float32)
preds_tensor = Tensor(preds)

bce_logit_loss_mean = BCEWithLogitsLoss(reduction='mean')
bce_logit_loss_sum = BCEWithLogitsLoss(reduction='sum')

loss_mean = bce_logit_loss_mean(preds_tensor, y)
loss_sum = bce_logit_loss_sum(preds_tensor, y)

print(f"Mean Loss: {loss_mean}")
print(f"Sum Loss: {loss_sum}")

Mean Loss: tensor: 33.56257629394531
Sum Loss: tensor: 1074.00244140625


In [67]:
# Manual loss calculation
sigmoid_output = 1 / (1 + np.exp(-preds))
log_1 = np.log(sigmoid_output).astype(np.float32)
log_2 = np.log(1 - sigmoid_output).astype(np.float32)
log_1 = np.clip(log_1, -100, float('inf'))
log_2 = np.clip(log_2, -100, float('inf'))
loss_term_1 = - targets * log_1
loss_term_2 = - (1 - targets) * log_2
loss = loss_term_1 + loss_term_2
loss_manual_mean = np.mean(loss)
loss_manual_sum = np.sum(loss)

print(f"Manual Mean Loss: {loss_manual_mean}")
print(f"Manual Sum Loss: {loss_manual_sum}")

Manual Mean Loss: 33.56257629394531
Manual Sum Loss: 1074.00244140625


In [77]:
# Check loss values
np.equal(loss_mean.data, loss_manual_mean), np.equal(loss_sum.data, loss_manual_sum)

(True, True)

In [78]:
# Forward pass
np.random.seed(0)

layer_1 = Linear(10, 5)
relu = ReLU()
layer_2 = Linear(5, 1)

# Forward pass
h1 = layer_1(x)
h2 = relu(h1)
out = layer_2(h2)

out.shape

(32, 1)

In [79]:
# Manual forward pass
weights_1 = layer_1.weight.data
bias_1 = layer_1.bias.data
weights_2 = layer_2.weight.data
bias_2 = layer_2.bias.data

h1_manual = np.dot(inputs, weights_1) + bias_1
h2_manual = np.maximum(h1_manual, 0)
h3_manual = np.dot(h2_manual, weights_2) + bias_2
out_manual = 1 / (1 + np.exp(-h3_manual))

out_manual.shape

(32, 1)

In [81]:
# Check if the outputs are the same
np.equal(Sigmoid()(out).data, out_manual).all()

True

In [82]:
# Loss
bce_loss = BCEWithLogitsLoss(reduction='mean')
loss_val = bce_loss(out, y)

# Manual loss
log_1 = np.log(out_manual).astype(np.float32)
sub_out_manual = 1 - out_manual
log_2 = np.log(sub_out_manual).astype(np.float32)
log_1_clip = np.clip(log_1, -100, float('inf'))
log_2_clip = np.clip(log_2, -100, float('inf'))
loss_term_1 = - targets * log_1_clip
loss_term_2 = - (1 - targets) * log_2_clip
loss = loss_term_1 + loss_term_2
loss_manual = np.mean(loss)

np.equal(loss_val.data, loss_manual).all()

True

In [83]:
# Backward pass
loss_val.backward()

In [87]:
# Manual backward pass
dL = np.ones_like(loss)
print((loss_val.grad == dL).all())

d_loss = dL / 32
print((d_loss == loss_val._prev[0].grad).all(), loss_val._prev[0].label)

d_loss_term_1 = d_loss
print((d_loss_term_1 == loss_val._prev[0]._prev[0].grad).all()) 

d_loss_term_2 = d_loss
print((d_loss_term_1 == loss_val._prev[0]._prev[1].grad).all()) 

d_log_1_clip = - targets * d_loss_term_1
print((d_log_1_clip == loss_val._prev[0]._prev[0]._prev[1].grad).all(), loss_val._prev[0]._prev[0]._prev[1].label) 

d_log_2_clip = d_loss_term_2 * - (1 - targets)
print((d_log_2_clip == loss_val._prev[0]._prev[1]._prev[1].grad).all(), loss_val._prev[0]._prev[1]._prev[1].label)

d_log_1  = d_log_1_clip * (log_1 > -100)
print((d_log_1 == loss_val._prev[0]._prev[0]._prev[1]._prev[0].grad).all(), loss_val._prev[0]._prev[0]._prev[1]._prev[0].label)

d_log_2 = d_log_2_clip * (log_2 > -100)
print((d_log_2 == loss_val._prev[0]._prev[1]._prev[1]._prev[0].grad).all(), loss_val._prev[0]._prev[1]._prev[1]._prev[0].label)

d_out_manual = d_log_1  / out_manual
d_sub_out_manual = d_log_2 / sub_out_manual
d_out_manual = d_out_manual - d_sub_out_manual

d_h3_manual = d_out_manual * out_manual * (1 - out_manual)  
print((d_h3_manual == out.grad).all())

d_weights_2 = np.dot(h2_manual.T, d_h3_manual)
d_bias_2 = np.sum(d_h3_manual, axis=0)
print((d_weights_2 == layer_2.weight.grad).all())

d_h2_manual = np.dot(d_h3_manual, weights_2.T)
print((d_h2_manual == h2.grad).all())

d_h1_manual = d_h2_manual * (h1_manual > 0)
print((d_h1_manual == h1.grad).all())

d_weights_1 = np.dot(inputs.T, d_h1_manual)
d_bias_1 = np.sum(d_h1_manual, axis=0)
print((d_weights_1 == layer_1.weight.grad).all())
print((d_bias_1 == layer_1.bias.grad).all())

True
True  *  * clamp(log(sigmoid(ReLU( @  + ) @  + ))) +  *  +  *  * clamp(log(sigmoid(ReLU( @  + ) @  + ) *  + ))
True
True
True clamp(log(sigmoid(ReLU( @  + ) @  + )))
True clamp(log(sigmoid(ReLU( @  + ) @  + ) *  + ))
True log(sigmoid(ReLU( @  + ) @  + ))
True log(sigmoid(ReLU( @  + ) @  + ) *  + )
True
True
True
True
True
True


### CrossEntropy Loss